In [2]:
from konlpy.tag import Twitter

twitter = Twitter()

In [5]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=181414&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'

import requests
from bs4 import BeautifulSoup


In [7]:
page=1 
response = requests.get(url.format(1))
response

<Response [200]>

In [12]:
soup = BeautifulSoup(response.text, 'html.parser') #html을 하나씩 하나씩 보는 것
soup.find_all('li')

[<li class="on"><a href="#" onclick="parent.clickcr(this, 'ara.bysym', '', '', event); dislplayOrder('sympathyScore');">공감순</a></li>,
 <li><a href="#" onclick="parent.clickcr(this, 'ara.byrct', '', '', event); dislplayOrder('newest');">최신순</a></li>,
 <li><a href="#" onclick="parent.clickcr(this, 'ara.high', '', '', event); dislplayOrder('highest');">평점 높은 순</a></li>,
 <li><a href="#" onclick="parent.clickcr(this, 'ara.low', '', '', event); dislplayOrder('lowest');">평점 낮은 순</a></li>,
 <li>
 <input class="blind" id="spoilerYnCheckBox" name="spilerViewer" onclick="parent.clickcr(this,'','','',event); return false;" title="스포일러 보기" type="checkbox"/>
 <label class="label_viewer" for="spoilerYnCheckBox" id="spoilerYnLable">스포일러 보기</label>
 </li>,
 <li>
 <input class="blind" id="actualYnCheckBox" name="viewer" onclick="parent.clickcr(this,'ura.mgs','','',event); return false;" title="관람객 평점만 보기" type="checkbox"/>
 <label class="label_viewer" for="actualYnCheckBox" id="actualYnLable">관람객 평점만 보

In [27]:
for li in soup.find('div', {'class': 'score_result'}).find_all('li'):
    print("점수:", li.em.text)
    print("댓글:", li.p.text.strip().replace("\t","").replace("\n","").replace("\r",""))
    

점수: 8
댓글: 관람객빨간옷중.. 유일하게 언어를 쓸수 있는 이유..
점수: 10
댓글: 공포영화인줄 알았는데 미친영화였다...
점수: 8
댓글: 영화 초반=컨저링급 후반=짱구의 좌충우돌 아메리카 대모험
점수: 6
댓글: 호불호 많이 갈릴듯 소재는 참신했으나 풀어나가는 능력이 다소 미흡했다. 겟아웃 정도의 참신함과 몰입력은 아닌거같다.
점수: 8
댓글: 어떤 영화가 낫다 공감 안낫다 비공감 이런것좀 하지마 맨날 이런게 베댓이야 어스자체만의 평점을 매기라고
점수: 10
댓글: 루피타 뇽 연기력 미쳤다...
점수: 10
댓글: 재밋게 봣는데 궁금한건 손에 손잡고 계속 잇는겨? 저거 하면 뭐 어케 되는건데
점수: 5
댓글: 관람객겟아웃 보다 확실히 별로입니다.
점수: 2
댓글: 관람객재밌냐 이게 ? 솔직히 이게 재밌냐고
점수: 3
댓글: 겟아웃보다 훨씬 별로인데 언플하고 홍보를 너무 해서 기대치를 지나치게 높임;;;


In [40]:
def get_reple(page=1): 
    response = requests.get(url.format(page))
    soup = BeautifulSoup(response.text, 'html.parser') 
    ## 댓글 수집 자동화
    s, t = [], []
    
    
    for li in soup.find('div', {'class': 'score_result'}).find_all('li'):
        #print("점수:", li.em.text)
        #print("댓글:", li.p.text.strip().replace("\t","").replace("\n","").replace("\r",""))
        if int(li.em.text) >= 8:
            s.append(1)
            t.append(li.p.text.strip().replace("\t","").replace("\n","").replace("\r",""))
        elif int(li.em.text) <= 5:
            s.append(0)
            t.append(li.p.text.strip().replace("\t","").replace("\n","").replace("\r",""))
    return s, t

In [41]:
get_reple(1)

([1, 1, 1, 1, 1, 1, 0, 0, 0],
 ['관람객빨간옷중.. 유일하게 언어를 쓸수 있는 이유..',
  '공포영화인줄 알았는데 미친영화였다...',
  '영화 초반=컨저링급 후반=짱구의 좌충우돌 아메리카 대모험',
  '어떤 영화가 낫다 공감 안낫다 비공감 이런것좀 하지마 맨날 이런게 베댓이야 어스자체만의 평점을 매기라고',
  '루피타 뇽 연기력 미쳤다...',
  '재밋게 봣는데 궁금한건 손에 손잡고 계속 잇는겨? 저거 하면 뭐 어케 되는건데',
  '관람객겟아웃 보다 확실히 별로입니다.',
  '관람객재밌냐 이게 ? 솔직히 이게 재밌냐고',
  '겟아웃보다 훨씬 별로인데 언플하고 홍보를 너무 해서 기대치를 지나치게 높임;;;'])

In [42]:
# 네이버 도스 공격 한다고 생각하고 막을수 있음
import time
score, text = [], []

for i in range(1,1000):
    time.sleep(1)
    print(i, end = "\r") 
    s, t = get_reple(i)
    score += s
    text += t

In [43]:
len(score), len(text)

8464

In [45]:
import pandas as pd

df = pd.DataFrame([score, text]).T
df.columns = ['score', 'text']

In [46]:
df.to_csv('text(us).csv')

In [47]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(text, score, test_size = 0.2, random_state=0)

In [48]:
len(train_x), len(train_y)

(6771, 6771)

In [49]:
len(test_x),len(test_y)

(1693, 1693)

In [50]:
from konlpy.tag import Twitter
twitter = Twitter()
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(tokenizer= twitter.morphs, ngram_range=(1,2), min_df= 3, max_df = 0.9) 
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x


C:\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


<6771x8882 sparse matrix of type '<class 'numpy.float64'>'
	with 122951 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state = 0)
params = {'C' : [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid = params, cv=4, scoring = 'accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.6s finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': [1, 3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [52]:
grid_cv.best_params_

{'C': 5}

In [53]:
grid_cv.best_score_

0.8604342046964998

In [54]:
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x, test_y)

0.861193148257531

In [56]:
a = ['아 너무 재미있어요', '핵노잼','꿀잼','노잼', '영화가 지루해요', '보지마삼']
my_review = tfv.transform(a)
grid_cv.best_estimator_.predict(my_review)

array([1, 0, 1, 0, 0, 0])